### Exp: 2 Cluster customers first... Then map each cluster to vehicle

Assumption: Customer comes in cluster big small medium with some noise(somebody crossing da road) clustering them might mean they comes from the same vehicle (or come together from somewhere)

Cross : Create a HDBSCAN cluster then tune it accordingly

Cross OUTPUT : 
- df_cross['cluster'] : the cluster that the that customer belongs to with 
- cluster_cross : the cluster info of cross containin 
    - xy mid mean, min , max - timestamp (start stop avg) and COUNT of cluster size


camera used : mbk-14-12-vehicle 

### Load

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger import Cluster

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

CURRENT_CAM = "mbk-14-12"

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-vehicle-object-20240428.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-cross-object-20240428.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-reverse-object-20240428.csv")
)

df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_NON_SNAPSHOT)
print(df_vehicle.to_string())
# print(df_vehicle[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT]].to_string())

      is_plate_valid  travel_distance  vehicle_type_confidence  lifetime  ymin plate_number  xmin  province_confidence similarized_plate_number  plate_number_confidence  xmax                                                                                      image_name purchasing_power province             camera  travel_displacement            timestamp   car_brand  travel_trajectory  frame_no               original_video_name  ymax  plate_color_confidence vehicle_type  car_brand_confidence plate_color  plate_image_name       timestamp_precise           true_video_name                        video_name    xmid   ymid  timestamp_unix  timestamp_unix_end
2315            True      2828.286892                 0.355878    14.236     0       327722  1221             0.951550                   327722                 0.993560  2191   2024-04-28T09-27-41_FLAG_P_FID_1291_OID_53g0_mbk-14-12-vehicle_1714271101_1714271261_1291.jpg          economy    th-10  mbk-14-12-vehicle           978.357679 

### Preprocess

In [2]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT,'timestamp_unix']].to_string())

In [3]:
df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

### Clustering the customer

In [4]:
df_cross ,  cluster_cross = Cluster.perform_cross_clustering(df_cross=df_cross)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT, 'cluster']].to_string())

x:1227.5, y:575.5, t:1714269608.0, cluster: -1
x:2126.0, y:46.5, t:1714270291.0, cluster: 0
x:547.5, y:813.5, t:1714270368.0, cluster: 0
x:250.5, y:816.0, t:1714271026.0, cluster: 28
x:250.5, y:802.5, t:1714271073.0, cluster: 28
x:2140.0, y:26.5, t:1714271110.0, cluster: 29
x:250.5, y:802.5, t:1714271124.0, cluster: 29
x:2133.0, y:31.5, t:1714271127.0, cluster: 29
x:2087.5, y:127.5, t:1714271238.0, cluster: 13
x:2115.5, y:92.5, t:1714271241.0, cluster: 13
x:2120.0, y:96.0, t:1714271244.0, cluster: 13
x:2138.0, y:66.5, t:1714271654.0, cluster: 5
x:2133.0, y:64.0, t:1714271660.0, cluster: 5
x:2128.5, y:45.5, t:1714271661.0, cluster: 5
x:1490.0, y:585.0, t:1714272269.0, cluster: 1
x:2011.5, y:160.0, t:1714272397.0, cluster: 1
x:2127.5, y:40.5, t:1714272523.0, cluster: 1
x:936.5, y:832.5, t:1714273394.0, cluster: 14
x:1101.5, y:736.0, t:1714273400.0, cluster: 14
x:1622.5, y:486.5, t:1714273401.0, cluster: 14
x:1057.0, y:682.0, t:1714273405.0, cluster: 14
x:1078.0, y:735.0, t:1714273409.0, 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

     cluster_id  xmid_min  xmid_max    xmid_mean  ymid_min  ymid_max   ymid_mean  timestamp_unix_min  timestamp_unix_max  timestamp_unix_mean  count
0            -1     280.5    2163.5  1650.337500      32.5     909.5  305.862500        1.714270e+09        1.714318e+09         1.714296e+09     40
1             0     547.5    2126.0  1336.750000      46.5     813.5  430.000000        1.714270e+09        1.714270e+09         1.714270e+09      2
2             1    1490.0    2127.5  1876.333333      40.5     585.0  261.833333        1.714272e+09        1.714273e+09         1.714272e+09      3
3             2     270.0    2124.5   967.666667      91.0     781.5  540.500000        1.714274e+09        1.714274e+09         1.714274e+09      3
4             3    1275.0    2120.0  1837.666667      79.5     565.0  242.000000        1.714281e+09        1.714282e+09         1.714281e+09      3
5             4     303.0    2132.0  1558.500000      45.5     974.0  371.416667        1.714289e+09      

TODO: find a better metric function to calculate distances